In [33]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import math
from math import log10
%matplotlib inline
start_time = pd.datetime.now()

In [34]:
df = pd.read_excel('card transactions.xlsx', converters={'Merchnum': lambda x: str(x)})
df.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [35]:
df.shape

(96753, 10)

In [36]:
df2 = df[~df['Merch description'].str.contains("FEDEX")].reset_index().drop('index',axis=1)
df2 = df2[~df2['Merch description'].str.contains("FED EX")].reset_index().drop('index',axis=1)
df2 = df2[~df2['Merch description'].str.contains("FEDERAL EXPRESS")].reset_index().drop('index',axis=1)
df2.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
1,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
2,8,5142191182,2010-01-01,6098208200062,MIAMI COMPUTER SUPPLY,OH,45429.0,P,230.32,0
3,9,5142258629,2010-01-01,602608969534,FISHER SCI ATL,GA,30091.0,P,62.11,0
4,14,5142124791,2010-01-01,5725000466504,CDW*GOVERNMENT INC,IL,60061.0,P,106.89,0


In [37]:
df2.shape

(84909, 10)

In [ ]:
result1['n'] = result1[['cat_x','cat_y']].map(lambda x: result1.cat_x if result1.cat_y == np.nan else(result1.cat_y if result1.cat_x==np.nan else result1.cat_x + result1.cat_y))

In [46]:
df2['nlowhigh'] = pd.Series(map(lambda x: 'nhigh' if int(str(x)[0])>2 else
                                ('nlow' if int(str(x)[0])>0 else ('nhigh' if int(str(x)[2])>2 else 'nlow')), df2.Amount))
df2.head(10)

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,nlowhigh
0,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,nhigh
1,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,nlow
2,8,5142191182,2010-01-01,6098208200062,MIAMI COMPUTER SUPPLY,OH,45429.0,P,230.32,0,nlow
3,9,5142258629,2010-01-01,602608969534,FISHER SCI ATL,GA,30091.0,P,62.11,0,nhigh
4,14,5142124791,2010-01-01,5725000466504,CDW*GOVERNMENT INC,IL,60061.0,P,106.89,0,nlow
5,15,5142151402,2010-01-01,5725000466504,CDW*GOVERNMENT INC,IL,60061.0,P,137.45,0,nlow
6,20,5142190596,2010-01-01,602608969284,FISHER SCI HUS,TX,77251.0,P,218.57,0,nlow
7,24,5142217905,2010-01-01,4353000719908,AMAZON.COM *SUPERSTRE,WA,98101.0,P,68.90,0,nhigh
8,27,5142183817,2010-01-01,5725000466504,CDW*GOVERNMENT INC,IL,60061.0,P,360.00,0,nhigh
9,29,5142270480,2010-01-01,5725000466504,CDW*GOVERNMENT INC,IL,60061.0,P,427.00,0,nhigh


In [47]:
df2[df2['nlowhigh']=='nhigh'].groupby('Cardnum')['nlowhigh'].count()

Cardnum
5142110081     4
5142110313     1
5142110402     8
5142110651     3
5142110691    13
              ..
5142310347     3
5142310397     1
5142310525     3
5142310768     2
5142847398    35
Name: nlowhigh, Length: 1565, dtype: int64

In [40]:
dfN = df2.groupby('Cardnum')['nlowhigh'].count()
dfN = pd.DataFrame({'Cardnum':dfN.index, 'n':dfN.values})
dfL = df2[df2['nlowhigh']=='nlow'].groupby('Cardnum')['nlowhigh'].count()
dfL = pd.DataFrame({'Cardnum':dfL.index, 'nlow':dfL.values})
dfH = df2[df2['nlowhigh']=='nhigh'].groupby('Cardnum')['nlowhigh'].count()
dfH = pd.DataFrame({'Cardnum':dfH.index, 'nhigh':dfH.values})
Cjoin = pd.merge(pd.merge(dfN, dfL, on ='Cardnum', how ='outer'), dfH, on ='Cardnum', how ='outer')
Cjoin.fillna(1, inplace=True)
Cjoin['nlow'],Cjoin['nhigh']=Cjoin['nlow'].astype(int),Cjoin['nhigh'].astype(int)
Cjoin

,Cardnum,n,nlow,nhigh
0,5142110002,1,1,1
1,5142110081,4,1,4
2,5142110313,3,2,1
3,5142110402,13,5,8
4,5142110434,1,1,1
...,...,...,...,...
1637,5142310397,1,1,1
1638,5142310525,4,1,3
1639,5142310598,2,2,1
1640,5142310768,4,2,2


In [41]:
Cjoin['R'] = 1.096 * Cjoin['nlow'] / Cjoin['nhigh']
Cjoin['1/R'] = 1/Cjoin['R']
Cjoin['U'] = Cjoin[['R','1/R']].max(axis=1)
Cjoin['t'] = (Cjoin['n'] - 15)/3
Cjoin['U*'] = 1 + (Cjoin['U'] - 1)/(1 + np.exp(-Cjoin['t']))
Cjoin.sort_values(by=['U*'], ascending=False)

,Cardnum,n,nlow,nhigh,R,1/R,U,t,U*
1178,5142253356,66,61,5,13.371200,0.074788,13.371200,17.000000,13.371199
1542,5142299705,28,25,3,9.133333,0.109489,9.133333,4.333333,9.027976
725,5142197563,149,15,134,0.122687,8.150852,8.150852,44.666667,8.150852
150,5142127276,27,3,24,0.137000,7.299270,7.299270,4.000000,7.185970
697,5142194617,38,5,33,0.166061,6.021898,6.021898,7.666667,6.019548
...,...,...,...,...,...,...,...,...,...
331,5142150984,1,1,1,1.096000,0.912409,1.096000,-4.666667,1.000894
1394,5142282130,1,1,1,1.096000,0.912409,1.096000,-4.666667,1.000894
343,5142152186,1,1,1,1.096000,0.912409,1.096000,-4.666667,1.000894
1397,5142282308,1,1,1,1.096000,0.912409,1.096000,-4.666667,1.000894


In [42]:
dfN2 = df2.groupby('Merchnum')['nlowhigh'].count()
dfN2 = pd.DataFrame({'Merchnum':dfN2.index, 'n':dfN2.values})
dfL2 = df2[df2['nlowhigh']=='nlow'].groupby('Merchnum')['nlowhigh'].count()
dfL2 = pd.DataFrame({'Merchnum':dfL2.index, 'nlow':dfL2.values})
dfH2 = df2[df2['nlowhigh']=='nhigh'].groupby('Merchnum')['nlowhigh'].count()
dfH2 = pd.DataFrame({'Merchnum':dfH2.index, 'nhigh':dfH2.values})
Mjoin = pd.merge(pd.merge(dfN2, dfL2, on ='Merchnum', how ='outer'), dfH2, on ='Merchnum', how ='outer')
Mjoin.fillna(1, inplace=True)
Mjoin['nlow'],Mjoin['nhigh']=Mjoin['nlow'].astype(int),Mjoin['nhigh'].astype(int)
Mjoin

,Merchnum,n,nlow,nhigh
0,0,230,118,112
1,003100006NOT6,1,1,1
2,004740006ABC6,1,1,1
3,005590006PNB6,1,1,1
4,014430619 14,1,1,1
...,...,...,...,...
13058,DU49038320006,1,1,1
13059,JCPENNE9 CO,3,1,2
13060,PENNE9 CO #05,1,1,1
13061,PENNE9 CO #68,1,1,1


In [43]:
Mjoin['R'] = 1.096 * Mjoin['nlow'] / Mjoin['nhigh']
Mjoin['1/R'] = 1/Mjoin['R']
Mjoin['U'] = Mjoin[['R','1/R']].max(axis=1)
Mjoin['t'] = (Mjoin['n'] - 15)/3
Mjoin['U*'] = 1 + (Mjoin['U'] - 1)/(1 + np.exp(-Mjoin['t']))
Mjoin.sort_values(by=['U*'], ascending=False)

,Merchnum,n,nlow,nhigh,R,1/R,U,t,U*
12821,991808369338,181,1,181,0.006055,165.145985,165.145985,55.333333,165.145985
9812,8078200641472,60,59,1,64.664000,0.015465,64.664000,15.000000,64.663981
1949,308904389335,53,1,53,0.020679,48.357664,48.357664,12.666667,48.357515
2348,3523000628102,34,34,1,37.264000,0.026836,37.264000,6.333333,37.199706
9905,808998385332,37,1,36,0.030444,32.846715,32.846715,7.333333,32.825921
...,...,...,...,...,...,...,...,...,...
7362,663104212,1,1,1,1.096000,0.912409,1.096000,-4.666667,1.000894
7361,663101800,1,1,1,1.096000,0.912409,1.096000,-4.666667,1.000894
2327,35068812332,1,1,1,1.096000,0.912409,1.096000,-4.666667,1.000894
13062,R7071969,1,1,1,1.096000,0.912409,1.096000,-4.666667,1.000894
